# EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv('./data/wsl/avg_scores_all_years.csv', index_col=0)
df.head()

,year,event,round,heat,name,top_two_waves_total
0,2008,Quiksilver Pro Gold Coast,Round 2,Heat 1,Andy Irons,16.37
1,2008,Quiksilver Pro Gold Coast,Round 2,Heat 1,Tamaroa McComb,5.90
2,2008,Quiksilver Pro Gold Coast,Round 2,Heat 2,Julian Wilson,14.00
3,2008,Quiksilver Pro Gold Coast,Round 2,Heat 2,Pancho Sullivan,13.23
4,2008,Quiksilver Pro Gold Coast,Round 2,Heat 3,Jeremy Flores,10.83


In [10]:
df['event'].unique()

array(['Quiksilver Pro Gold Coast', 'Rip Curl Pro Bells Beach',
       'Billabong Pro Teahupoo', 'Globe Pro Fiji', 'Billabong Pro J-Bay',
       'Rip Curl Pro Search Bali', 'Boost Mobile Pro',
       'Quiksilver Pro France', 'Billabong Pro Mundaka',
       'Hang Loose Santa Catarina Pro', 'Billabong Pipeline Masters',
       'Hurley Pro Trestles', 'Rip Curl Search', 'Santa Catarina Pro',
       'Rip Curl Pro Portugal', 'Billabong Pipe Masters',
       'Billabong Rio Pro', 'Quiksilver Pro New York', 'Volcom Fiji Pro',
       "O'Neill Coldwater Classic Santa Cruz", 'Oakley Pro Bali',
       'Drug Aware Margaret River Pro', 'Fiji Pro', 'J-Bay Open',
       'Billabong Pro Tahiti', 'Hurley Pro at Trestles',
       'Moche Rip Curl Pro Portugal', 'Oi Rio Pro',
       'MEO Rip Curl Pro Portugal', 'Outerknown Fiji Pro',
       'Corona Open J-Bay', 'Corona Bali Protected',
       'Uluwatu CT Margaret River Pro', "Tahiti Pro Teahupo'o",
       'Margaret River Pro',
       'Billabong Pipe Masters 

In [11]:
pipe = [
    "Billabong Pipe Masters presented by Hydro Flask",
    "Billabong Pro Pipeline",
    "Billabong Pipe Masters presented by Hydro Flask",
    "Billabong Pipe Masters",
    "Billabong Pipeline Masters"
]

df.loc[df['event'].isin(pipe)].shape

(1649, 6)

In [13]:
df['event'].value_counts()

Quiksilver Pro Gold Coast                              1473
Quiksilver Pro France                                  1444
Rip Curl Pro Bells Beach                               1422
Billabong Pipe Masters                                 1176
Billabong Pro Teahupoo                                  729
Hurley Pro Trestles                                     614
Oi Rio Pro                                              574
Billabong Pro J-Bay                                     493
Hurley Pro at Trestles                                  472
Billabong Pro Tahiti                                    472
Billabong Rio Pro                                       472
Rip Curl Pro Portugal                                   472
Drug Aware Margaret River Pro                           471
MEO Rip Curl Pro Portugal                               456
J-Bay Open                                              354
Fiji Pro                                                354
Corona Open J-Bay                       

In [12]:
df['event'].value_counts()[0]

1473

**Pipeline is the event with most data**

In [14]:
df['year'] = df['year'].astype(str)

In [15]:
df['round'].unique()

array(['Round 2', 'Round 1', 'Round 3', 'Round 4', 'Quarterfinal',
       'Semifinal', 'Final', 'Round 5', 'Quarterfinals', 'Semifinals',
       'Trials Quarterfinals', 'Trials Round 1', 'Trials Semifinals',
       'Trials Final', 'Elimination Round', 'Seeding Round',
       'Round of 32', 'Round of 16', 'Opening Round'], dtype=object)

In [16]:
df["round"] = df["round"].map(
    {
        "Seeding Round": "Round 1",
        "Round of 32": "Round 3",
        "Quarterfinal": "Quarterfinals",
        "Round of 16": "Round 4",
        "Elimination Round": "Round 2",
        "Opening Round": "Round 1",
        "Round 1": "Round 1",
        "Round 2": "Round 2",
        "Round 3": "Round 3",
        "Round 4": "Round 4",
        "Round 5": "Round 5",
        "Quarterfinals": "Quarterfinals",
        "Final": "Final",
        "Semifinal": "Semifinals",
        "Semifinals": "Semifinals",
        "Trials Quarterfinals": "Trials Quarterfinals",
        "Trials Round 1": "Trials Round 1",
        "Trials Semifinals": "Trials Semifinals",
        "Trials Final": "Trials Final",
    }
)


In [17]:
df['heat'].unique()

array(['Heat 1 ', 'Heat 2 ', 'Heat 3 ', 'Heat 4 ', 'Heat 5 ', 'Heat 6 ',
       'Heat 7 ', 'Heat 8 ', 'Heat 9 ', 'Heat 10 ', 'Heat 11 ',
       'Heat 12 ', 'Heat 13 ', 'Heat 14 ', 'Heat 15 ', 'Heat 16 ',
       'Heat 8 Watch replay', 'Heat 6 Watch replay',
       'Heat 2 Watch replay', 'Heat 1 Watch replay',
       'Heat 4 Watch replay', 'Heat 10 Watch replay',
       'Heat 11 Watch replay', 'Heat 12 Watch replay',
       'Heat 3 Watch replay', 'Heat 5 Watch replay',
       'Heat 7 Watch replay', 'Heat 9 Watch replay',
       'Heat 13 Watch replay', 'Heat 14 Watch replay',
       'Heat 15 Watch replay', 'Heat 16 Watch replay'], dtype=object)

In [18]:
df["heat"] = df["heat"].map(
    {
        "Heat 1 ": "Heat 1",
        "Heat 2 ": "Heat 2",
        "Heat 3 ": "Heat 3",
        "Heat 4 ": "Heat 4",
        "Heat 5 ": "Heat 5",
        "Heat 6 ": "Heat 6",
        "Heat 7 ": "Heat 7",
        "Heat 8 ": "Heat 8",
        "Heat 9 ": "Heat 9",
        "Heat 10 ": "Heat 10",
        "Heat 11 ": "Heat 11",
        "Heat 12 ": "Heat 12",
        "Heat 13 ": "Heat 13",
        "Heat 14 ": "Heat 14",
        "Heat 15 ": "Heat 15",
        "Heat 16 ": "Heat 16",
        "Heat 8 Watch replay": "Heat 8",
        "Heat 6 Watch replay": "Heat 6",
        "Heat 2 Watch replay": "Heat 2",
        "Heat 1 Watch replay": "Heat 1",
        "Heat 4 Watch replay": "Heat 4",
        "Heat 10 Watch replay": "Heat 10",
        "Heat 11 Watch replay": "Heat 11",
        "Heat 12 Watch replay": "Heat 12",
        "Heat 3 Watch replay": "Heat 3",
        "Heat 5 Watch replay": "Heat 5",
        "Heat 7 Watch replay": "Heat 7",
        "Heat 9 Watch replay": "Heat 9",
        "Heat 13 Watch replay": "Heat 13",
        "Heat 14 Watch replay": "Heat 14",
        "Heat 15 Watch replay": "Heat 15",
        "Heat 16 Watch replay": "Heat 16",
    }
)


In [19]:
df_pipe = df.loc[df['event'].isin(pipe)].reindex()
df_pipe

,year,event,round,heat,name,top_two_waves_total
0,2008,Billabong Pipeline Masters,Round 2,Heat 1,Kamalei Alexander,11.90
1,2008,Billabong Pipeline Masters,Round 2,Heat 1,Mikael Picon,11.67
2,2008,Billabong Pipeline Masters,Round 2,Heat 2,Evan Valiere,14.57
3,2008,Billabong Pipeline Masters,Round 2,Heat 2,Mick Campbell,14.34
4,2008,Billabong Pipeline Masters,Round 2,Heat 3,Jamie O'Brien,15.00
...,...,...,...,...,...,...
105,2022,Billabong Pro Pipeline,Semifinals,Heat 1,Miguel Pupo,9.50
106,2022,Billabong Pro Pipeline,Semifinals,Heat 2,Seth Moniz,13.50
107,2022,Billabong Pro Pipeline,Semifinals,Heat 2,Caio Ibelli,6.33
108,2022,Billabong Pro Pipeline,Final,Heat 1,Kelly Slater,18.77


In [50]:
event_dates = pd.read_csv('./data/wsl/pipe_masters_heat_dates.csv', index_col=0)
event_dates.head()

,year,event,round,heat,date
0,2008,Billabong Pipeline Masters,Round 1,Heat 1,
1,2008,Billabong Pipeline Masters,Round 1,Heat 2,
2,2008,Billabong Pipeline Masters,Round 1,Heat 3,
3,2008,Billabong Pipeline Masters,Round 1,Heat 4,
4,2008,Billabong Pipeline Masters,Round 1,Heat 5,


In [51]:
event_dates['date'].value_counts()

                      330
 December 19, 2014     27
 February 1, 2022      24
 December 16, 2018     24
 December 17, 2017     24
 December 18, 2016     23
 December 17, 2020     22
 December 16, 2015     19
 January 29, 2022      16
 December 11, 2019     16
 December 19, 2016     16
 December 10, 2019     16
 December 12, 2018     15
 December 19, 2019     15
 December 18, 2017     15
 December 12, 2014     12
 December 13, 2018     12
 December 14, 2016     12
 December 9, 2020      12
 December 11, 2015     12
 December 20, 2020     12
 December 10, 2015     12
 December 13, 2014     12
 December 17, 2018     11
 December 11, 2017      9
 December 17, 2015      8
 February 5, 2022       7
 December 16, 2017      3
 December 14, 2020      1
Name: date, dtype: int64

In [52]:
event_dates[event_dates['year'] == 2022]['round'].value_counts()

Round of 32          16
Opening Round        12
Round of 16           8
Elimination Round     4
Quarterfinals         4
Semifinals            2
Final                 1
Name: round, dtype: int64

In [53]:
event_dates['round'].unique()

array(['Round 1', 'Round 2', 'Round 3', 'Round 4', 'Quarterfinal',
       'Semifinal', 'Final', 'Round 5', 'Quarterfinals', 'Semifinals',
       'Trials Round 1', 'Trials Quarterfinals', 'Trials Semifinals',
       'Trials Final', 'Seeding Round', 'Elimination Round',
       'Round of 32', 'Round of 16', 'Opening Round'], dtype=object)

In [54]:
df_pipe['round'].unique()

array(['Round 2', 'Round 1', 'Round 3', 'Round 4', 'Quarterfinals',
       'Semifinals', 'Final', 'Round 5', 'Trials Quarterfinals',
       'Trials Round 1', 'Trials Semifinals', 'Trials Final'],
      dtype=object)

In [55]:
event_dates["round"] = event_dates["round"].map(
    {
        "Seeding Round": "Round 1",
        "Round of 32": "Round 3",
        "Quarterfinal": "Quarterfinals",
        "Round of 16": "Round 4",
        "Elimination Round": "Round 2",
        "Opening Round": "Round 1",
        "Round 1": "Round 1",
        "Round 2": "Round 2",
        "Round 3": "Round 3",
        "Round 4": "Round 4",
        "Round 5": "Round 5",
        "Quarterfinals": "Quarterfinals",
        "Final": "Final",
        "Semifinal": "Semifinals",
        "Semifinals": "Semifinals",
        "Trials Quarterfinals": "Trials Quarterfinals",
        "Trials Round 1": "Trials Round 1",
        "Trials Semifinals": "Trials Semifinals",
        "Trials Final": "Trials Final",
    }
)


In [56]:
event_dates['heat'].unique()

array(['Heat 1', 'Heat 2', 'Heat 3', 'Heat 4', 'Heat 5', 'Heat 6',
       'Heat 7', 'Heat 8', 'Heat 9', 'Heat 10', 'Heat 11', 'Heat 12',
       'Heat 13', 'Heat 14', 'Heat 15', 'Heat 16'], dtype=object)

In [57]:
df_pipe['heat'].unique()

array(['Heat 1', 'Heat 2', 'Heat 3', 'Heat 4', 'Heat 5', 'Heat 6',
       'Heat 7', 'Heat 8', 'Heat 9', 'Heat 10', 'Heat 11', 'Heat 12',
       'Heat 13', 'Heat 14', 'Heat 15', 'Heat 16'], dtype=object)

In [58]:
df_pipe['heat'].unique() == event_dates['heat'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [59]:
df_pipe['round'].unique() == event_dates['round'].unique()

array([False, False,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True])

In [60]:
event_dates['date'].unique()

array([' ', ' December 12, 2014', ' December 13, 2014',
       ' December 19, 2014', ' December 10, 2015', ' December 11, 2015',
       ' December 16, 2015', ' December 17, 2015', ' December 14, 2016',
       ' December 18, 2016', ' December 19, 2016', ' December 11, 2017',
       ' December 16, 2017', ' December 17, 2017', ' December 18, 2017',
       ' December 12, 2018', ' December 13, 2018', ' December 16, 2018',
       ' December 17, 2018', ' December 10, 2019', ' December 11, 2019',
       ' December 19, 2019', ' December 9, 2020', ' December 14, 2020',
       ' December 17, 2020', ' December 20, 2020', ' January 29, 2022',
       ' February 1, 2022', ' February 5, 2022'], dtype=object)

In [64]:
event_dates['date'] = event_dates['date'].str[1:]

In [68]:
event_dates = event_dates[event_dates['date'] != '']

In [71]:
event_dates['date'] = pd.to_datetime(event_dates['date'], infer_datetime_format=True)
event_dates['date'].unique()

array(['2014-12-12T00:00:00.000000000', '2014-12-13T00:00:00.000000000',
       '2014-12-19T00:00:00.000000000', '2015-12-10T00:00:00.000000000',
       '2015-12-11T00:00:00.000000000', '2015-12-16T00:00:00.000000000',
       '2015-12-17T00:00:00.000000000', '2016-12-14T00:00:00.000000000',
       '2016-12-18T00:00:00.000000000', '2016-12-19T00:00:00.000000000',
       '2017-12-11T00:00:00.000000000', '2017-12-16T00:00:00.000000000',
       '2017-12-17T00:00:00.000000000', '2017-12-18T00:00:00.000000000',
       '2018-12-12T00:00:00.000000000', '2018-12-13T00:00:00.000000000',
       '2018-12-16T00:00:00.000000000', '2018-12-17T00:00:00.000000000',
       '2019-12-10T00:00:00.000000000', '2019-12-11T00:00:00.000000000',
       '2019-12-19T00:00:00.000000000', '2020-12-09T00:00:00.000000000',
       '2020-12-14T00:00:00.000000000', '2020-12-17T00:00:00.000000000',
       '2020-12-20T00:00:00.000000000', '2022-01-29T00:00:00.000000000',
       '2022-02-01T00:00:00.000000000', '2022-02-05

In [74]:
event_dates[event_dates['date'] == '2014-12-12T00:00:00.000000000']

,year,event,round,heat,date
0,2014,Billabong Pipe Masters,Round 1,Heat 1,2014-12-12
1,2014,Billabong Pipe Masters,Round 1,Heat 2,2014-12-12
2,2014,Billabong Pipe Masters,Round 1,Heat 3,2014-12-12
3,2014,Billabong Pipe Masters,Round 1,Heat 4,2014-12-12
4,2014,Billabong Pipe Masters,Round 1,Heat 5,2014-12-12
5,2014,Billabong Pipe Masters,Round 1,Heat 6,2014-12-12
6,2014,Billabong Pipe Masters,Round 1,Heat 7,2014-12-12
7,2014,Billabong Pipe Masters,Round 1,Heat 8,2014-12-12
8,2014,Billabong Pipe Masters,Round 1,Heat 9,2014-12-12
9,2014,Billabong Pipe Masters,Round 1,Heat 10,2014-12-12
